<a href="https://colab.research.google.com/github/jeongukjae/distilmbert-for-representation/blob/main/knowledge_distillation_mse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "UNCOMPRESSED"
print(os.environ['COLAB_TPU_ADDR'])

10.69.13.122:8470


In [ ]:
!pip install -U -q tensorflow-text tensorflow-datasets

In [ ]:
HIDDEN_SIZE = 768

LEARNING_RATE = 8e-4
BATCH_SIZE = 1024
STEPS_PER_EPOCH = 1000
EPOCHS = 10
WARMUP_RATE = 0.05

In [ ]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
strategy = tf.distribute.TPUStrategy(cluster_resolver)

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.69.13.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.13.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
def create_student_preprocessing_model():
    return hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_preprocess/2")

def create_teacher_preprocessing_model():
    return hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")

In [ ]:
def create_student_model():
    encoder = hub.KerasLayer(
        "https://tfhub.dev/jeongukjae/distilbert_multi_cased_L-6_H-768_A-12/1",
        trainable=True,
    )
    inputs = {
        "input_word_ids": tf.keras.Input([None], dtype=tf.int32, name="input_word_ids"),
        "input_mask": tf.keras.Input([None], dtype=tf.int32, name="input_mask"),
    }
    logit = encoder(inputs)['pooled_output']
    logit = tf.keras.layers.Dense(HIDDEN_SIZE)(logit)
    model = tf.keras.Model(inputs, logit)
    model.summary()
    return model

def create_teacher_model():
    encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")
    inputs = {
        "input_word_ids": tf.keras.Input([None], dtype=tf.int32, name="input_word_ids"),
        "input_type_ids": tf.keras.Input([None], dtype=tf.int32, name="input_type_ids"),
        "input_mask": tf.keras.Input([None], dtype=tf.int32, name="input_mask"),
    }
    logit = encoder(inputs)['default']
    model = tf.keras.Model(inputs, logit)
    model.summary()
    return model

In [ ]:
class ModelForKD(tf.keras.Model):
    def __init__(self, model1, model2, **kwargs):
        super().__init__(**kwargs)
        self.model1 = model1
        self.model2 = model2

    def train_step(self, data):
        model1_input, model2_input = data

        model2_output = self.model2(model2_input)
        model2_output = tf.stop_gradient(model2_output)
        with tf.GradientTape() as tape:
            model1_output = self.model1(model1_input, training=True)
            loss = self.compiled_loss(model2_output, model1_output, regularization_losses=self.losses)

        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
        self.compiled_metrics.update_state(model2_output, model1_output)
        return {m.name: m.result() for m in self.metrics}

In [ ]:
class BertScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, rate, warmup_ratio, total_steps, name=None):
        super().__init__()

        self.rate = rate
        self.warmup_ratio = warmup_ratio
        self.total_steps = float(total_steps)
        self.warmup_steps = warmup_ratio * total_steps
        self.name = name

    def __call__(self, step):
        with tf.name_scope("BertScheduler"):
            total_steps = tf.convert_to_tensor(self.total_steps, name="total_steps")
            warmup_steps = tf.convert_to_tensor(self.warmup_steps, name="warmup_steps")

            current_step = step + 1.0

            return self.rate * tf.cond(
                current_step < warmup_steps,
                lambda: self.warmup(current_step, warmup_steps),
                lambda: self.decay(current_step, total_steps, warmup_steps),
            )

    @tf.function
    def warmup(self, step, warmup_steps):
        return step / tf.math.maximum(tf.constant(1.0), warmup_steps)

    @tf.function
    def decay(self, step, total_steps, warmup_steps):
        return tf.math.maximum(
            tf.constant(0.0), (total_steps - step) / tf.math.maximum(tf.constant(1.0), total_steps - warmup_steps)
        )

    def get_config(self):
        return {
            "warmup_ratio": self.warmup_ratio,
            "total_steps": self.total_steps,
            "warmup_steps": self.warmup_steps,
            "name": self.name,
        }

In [ ]:
LANGS = ['ar', 'bg', 'de', 'el', 'en', 'es', 'fr', 'hi', 'ru', 'sw', 'th', 'tr', 'ur', 'vi', 'zh']
NUM_LANGS = len(LANGS)


def get_xnli_dataset(student_preprocessor, teacher_preprocessor, batch_size, tfds_name="xtreme_xnli"):
    with tf.device('/job:localhost'):
        # batch_size=-1 is a way to load the dataset into memory
        in_memory_ds = tfds.load(tfds_name, split='train', batch_size=-1, shuffle_files=True)

    tfds_info = tfds.builder(tfds_name).info
    num_examples = tfds_info.splits['train'].num_examples * NUM_LANGS * 2
    train_ds = (
        tf.data.Dataset.from_tensor_slices(in_memory_ds)
        .flat_map(
            lambda x: tf.data.Dataset.from_tensor_slices(
                tf.concat([
                    tf.reshape(x['hypothesis']['translation'], [-1]),
                    tf.stack([x['premise'][lang] for lang in LANGS], axis=0),
                ], axis=0)
            )
        )
        .shuffle(num_examples, reshuffle_each_iteration=True)
        .batch(batch_size)
        .map(lambda x: (student_preprocessor(x), teacher_preprocessor(x)))
    )
    return train_ds, num_examples

In [ ]:
student_preprocessor = create_student_preprocessing_model()
teacher_preprocessor = create_teacher_preprocessing_model()

In [ ]:
import math


with strategy.scope():
    train_ds, num_examples = get_xnli_dataset(student_preprocessor, teacher_preprocessor, BATCH_SIZE)
    print("Num examples:", num_examples)
    steps_per_epoch = STEPS_PER_EPOCH  # math.ceil(num_examples / BATCH_SIZE)
    print("steps per epoch:", steps_per_epoch)
    num_train_steps = steps_per_epoch * EPOCHS
    print("total num steps:", num_train_steps)

    student_model = create_student_model()
    teacher_model = create_teacher_model()
    model = ModelForKD(student_model, teacher_model)
    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=['cosine_similarity'],
        optimizer=tf.keras.optimizers.Adam(learning_rate=BertScheduler(LEARNING_RATE, WARMUP_RATE, num_train_steps)),
    )

    model.fit(
        train_ds.repeat().prefetch(tf.data.AUTOTUNE),
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=EPOCHS,
    )

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Num examples: 11777100
steps per epoch: 1000
total num steps: 10000
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_mask (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
input_word_ids (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'pooled_output': (N 134734080   input_mask[0][0]                 
                                                                 input_word_ids[0][0]             
__________________________________________________________________________________________________
dense (Dense)             

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/PartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/PartitionedCall:0", shape=(None, 768), dtype=float32), dense_shape=Tensor("Adam/gradients/PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


1000/1000 [==============================] - 558s 461ms/step - loss: 0.1038 - cosine_similarity: 0.7361
Epoch 2/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0440 - cosine_similarity: 0.9024
Epoch 3/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0361 - cosine_similarity: 0.9209
Epoch 4/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0321 - cosine_similarity: 0.9301
Epoch 5/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0294 - cosine_similarity: 0.9363
Epoch 6/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0274 - cosine_similarity: 0.9409
Epoch 7/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0257 - cosine_similarity: 0.9446
Epoch 8/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0243 - cosine_similarity: 0.9479
Epoch 9/10
1000/1000 [==============================] - 460s 460ms/step - loss: 0.0231 - co

In [ ]:
MODEL_PATH = "./model/distilmbert-for-representation"

# Save everything on the Colab host (even the variables from TPU memory)
save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
student_model.save(MODEL_PATH, include_optimizer=False, options=save_options)

INFO:tensorflow:Assets written to: ./model/distilmbert-for-representation/assets


INFO:tensorflow:Assets written to: ./model/distilmbert-for-representation/assets


In [ ]:
!zip -r model.zip model/distilmbert-for-representation

updating: model/distilmbert-for-representation/ (stored 0%)
updating: model/distilmbert-for-representation/keras_metadata.pb (deflated 86%)
updating: model/distilmbert-for-representation/assets/ (stored 0%)
updating: model/distilmbert-for-representation/variables/ (stored 0%)
updating: model/distilmbert-for-representation/variables/variables.index (deflated 79%)
updating: model/distilmbert-for-representation/variables/variables.data-00000-of-00001 (deflated 7%)
updating: model/distilmbert-for-representation/saved_model.pb (deflated 92%)


In [ ]:
#!/bin/bash

# Langs:
# French, Spanish, German, Greek, Bulgarian,
# Russian, Turkish, Arabic, Vietnamese, Thai,
# Chinese, Hindi, Swahili, Urdu
!rm -rf data/tatoeba/v1
!mkdir -p data/tatoeba/v1
LANGS_TO_EVALUATE = [
    "fra", "spa", "deu", "ell", "bul",
    "rus", "tur", "ara", "vie", "tha",
    "cmn", "hin", "urd"]

for lang in LANGS_TO_EVALUATE:
    !curl -L "https://raw.githubusercontent.com/facebookresearch/LASER/2aa9cf8242f1030282be23a9cfa906fd011c4b2d/data/tatoeba/v1/tatoeba.{lang}-eng.{lang}" -o "./data/tatoeba/v1/tatoeba.{lang}-eng.{lang}"
    !curl -L "https://raw.githubusercontent.com/facebookresearch/LASER/2aa9cf8242f1030282be23a9cfa906fd011c4b2d/data/tatoeba/v1/tatoeba.{lang}-eng.eng" -o "./data/tatoeba/v1/tatoeba.{lang}-eng.eng"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43727  100 43727    0     0   261k      0 --:--:-- --:--:-- --:--:--  261k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36291  100 36291    0     0   217k      0 --:--:-- --:--:-- --:--:--  217k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37490  100 37490    0     0   245k      0 --:--:-- --:--:-- --:--:--  244k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34776  100 34776    0     0   246k      0 --:--:-- --:--:-- --:--:--  246k
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
from absl import logging
from tqdm import tqdm

@tf.function
def encode_sentences_teacher(s):
    return tf.nn.l2_normalize(teacher_model(teacher_preprocessor(s)), axis=-1)

@tf.function
def encode_sentences_student(s):
    return tf.nn.l2_normalize(student_model(student_preprocessor(s)), axis=-1)

for lang in LANGS_TO_EVALUATE:
    with tf.device('/job:localhost'):
        ds = [
            (src.numpy(), tgt.numpy()) for src, tgt in tf.data.Dataset.zip((
                tf.data.TextLineDataset([f"./data/tatoeba/v1/tatoeba.{lang}-eng.eng"]).batch(128),
                tf.data.TextLineDataset([f"./data/tatoeba/v1/tatoeba.{lang}-eng.{lang}"]).batch(128),
            ))
        ]

    src_embeddings_student = []
    tgt_embeddings_student = []

    src_embeddings_teacher = []
    tgt_embeddings_teacher = []
    for src, tgt in tqdm(ds):
        src_embeddings_student.append(encode_sentences_student(src).numpy())
        tgt_embeddings_student.append(encode_sentences_student(tgt).numpy())
        src_embeddings_teacher.append(encode_sentences_teacher(src).numpy())
        tgt_embeddings_teacher.append(encode_sentences_teacher(tgt).numpy())

    def print_score(srs, tgt):
        src_embeddings = tf.concat(srs, axis=0)
        tgt_embeddings = tf.concat(tgt, axis=0)

        similarities_src_to_tgt = tf.tensordot(src_embeddings, tgt_embeddings, axes=[[1], [1]])
        answer = tf.range(tf.shape(src_embeddings)[0], dtype=tf.int64)
        src_to_tgt_acc = tf.math.count_nonzero(tf.argmax(similarities_src_to_tgt, axis=-1) == answer) / tf.size(answer, tf.int64)
        tgt_to_src_acc = tf.math.count_nonzero(tf.argmax(tf.transpose(similarities_src_to_tgt, perm=[1, 0]), axis=-1) == answer) / tf.size(answer, tf.int64)
        # Print accuracy
        print(f"eng->{lang}:", src_to_tgt_acc, f", {lang}->eng:", tgt_to_src_acc)

    print("Student")
    print_score(src_embeddings_student, tgt_embeddings_student)
    print("Teacher")
    print_score(src_embeddings_teacher, tgt_embeddings_teacher)

100%|██████████| 8/8 [01:28<00:00, 11.02s/it]


Student
eng->fra: tf.Tensor(0.951, shape=(), dtype=float64) , fra->eng: tf.Tensor(0.953, shape=(), dtype=float64)
Teacher
eng->fra: tf.Tensor(0.959, shape=(), dtype=float64) , fra->eng: tf.Tensor(0.96, shape=(), dtype=float64)


100%|██████████| 8/8 [01:22<00:00, 10.27s/it]


Student
eng->spa: tf.Tensor(0.982, shape=(), dtype=float64) , spa->eng: tf.Tensor(0.985, shape=(), dtype=float64)
Teacher
eng->spa: tf.Tensor(0.981, shape=(), dtype=float64) , spa->eng: tf.Tensor(0.988, shape=(), dtype=float64)


100%|██████████| 8/8 [01:21<00:00, 10.24s/it]


...

Student
eng->cmn: tf.Tensor(0.955, shape=(), dtype=float64) , cmn->eng: tf.Tensor(0.94, shape=(), dtype=float64)
Teacher
eng->cmn: tf.Tensor(0.961, shape=(), dtype=float64) , cmn->eng: tf.Tensor(0.963, shape=(), dtype=float64)


100%|██████████| 8/8 [01:21<00:00, 10.24s/it]


Student
eng->hin: tf.Tensor(0.966, shape=(), dtype=float64) , hin->eng: tf.Tensor(0.956, shape=(), dtype=float64)
Teacher
eng->hin: tf.Tensor(0.979, shape=(), dtype=float64) , hin->eng: tf.Tensor(0.976, shape=(), dtype=float64)


100%|██████████| 8/8 [01:22<00:00, 10.25s/it]

Student
eng->urd: tf.Tensor(0.94, shape=(), dtype=float64) , urd->eng: tf.Tensor(0.934, shape=(), dtype=float64)
Teacher
eng->urd: tf.Tensor(0.96, shape=(), dtype=float64) , urd->eng: tf.Tensor(0.947, shape=(), dtype=float64)
